## Reconstrucción

In [1]:
#Librerias
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statistics as st

In [2]:
path ='/media/anmrodriguezsa/DA4E639F4E6372E5/Users/Ángela/Desktop/Proyecto1/acciones'  ##Linux
#path ='C:/Users/Ángela/Desktop/Proyecto1/acciones'   #windows

In [3]:
# Lectura de archivos .csv y Selección de los precios de cierre de cada acción.

def LecYSel(path):
    allFiles = glob.glob(path + "/*.csv")
    Pc=np.asarray([])
    flagFirst = 0
        
    for file_ in allFiles:
        Data = pd.read_csv(file_,index_col=None, header=0)  #Lectura de archivos .csv
        x  = Data["Último"].values   #Selección de los valores del precio de cierre de las acciones
        x = [s. replace('.','') for s in x] 
        x = [s. replace(',','.') for s in x]
        v = [float(s) for s in x] 
        closingPrice = np.flipud(v)
        
        if flagFirst==0:
            flagFirst = 1
            Pc = closingPrice
        else:
            Pc = np.vstack((Pc,closingPrice)) #"transpone"
    return(Pc)

In [4]:
#Cálculo promedio, desviación estándar, matriz de covarianzas de los precios de cierre de cada acción

def Av(X):                      
    Av=np.zeros(X.shape[0])           
    
    for i in range(X.shape[0]):
        Av[i]= st.mean(X[i,:])        #Cálculo de la media 
    return Av

def Dst(X):                      
    DS=np.zeros(X.shape[0])           
    
    for i in range(X.shape[0]):
        DS[i]= st.stdev(X[i,:])       #Cálculo de Desviación Estándar
    return DS

def cov(a, b):
    if len(a) != len(b):
        return

    a_mean = np.mean(a)
    b_mean = np.mean(b)
    sum = 0

    for i in range(0, len(a)):
        sum += ((a[i] - a_mean) * (b[i] - b_mean))     #Cálculo covarianza entre dos vectores
    return sum/(len(a)-1)


def Mcov(M):
    C=np.zeros((M.shape[0],M.shape[0]))
 
    for i in range(M.shape[0]):
        for j in range(M.shape[0]):
            C[i][j] = cov(M[i,:],M[j,:])                            #Matriz de covarianza
    return(C)

In [13]:
from scipy.optimize import minimize

#valores iniciales
r = 0
w0=np.array([1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])

def miniz(M,Pc):
    w=np.zeros(M.shape[0])
    for i in range(w.size):
        for j in range(w.size):
            S=(1/2)*sum(sum(M[i][j]*w[i]*w[j]))
    return S

def constraint1(Pc,r):
    U=np.ones(Pc.shape[0])
    w=np.zeros(Pc.shape[0])
    sum_= r
    for i in range(w.size):
        sum_=sum_ - Av[i]*w[i]        
    return sum_

def constraint2(Pc):
    w=np.zeros(Pc.shape[0])
    Sum_= 1
    for i in range(w.size):
        Sum_= Sum_ - w[i]        
    return Sum_

def constraint3(Pc):
    w=np.zeros(Pc.shape[0])    
    for i in range(w.size):
        w[i]>=0        
    return w

# Optimización
b = (0,1)
bnds = (b, b, b, b, b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b,b)
con1 = {'type': 'eq', 'fun': constraint1} 
con2 = {'type': 'eq', 'fun': constraint2}
con3 = {'type': 'ineq', 'fun': constraint3} 
cons = ([con1,con2,con3])
solution = minimize(miniz,w0,method='SLSQP',\
                    bounds=bnds,constraints=cons)
x = solution.x



TypeError: constraint1() missing 1 required positional argument: 'r'

In [22]:
#Simulación del cálculo de los pesos con el módelo clásico de Markowitz

def  wM(rf,Pc):
    M= Mcov(Pc)
    U= np.ones(Pc.shape[0])
    A=np.matmul(np.transpose(U),np.linalg.inv(M),Av(Pc))
    B=np.matmul(np.transpose(U),np.linalg.inv(M),U)
    C=np.matmul(np.transpose(Av(Pc)),np.linalg.inv(M),Av(Pc))
    D=B*C-A**2
    g=(1/D)*(B*np.matmul(np.linalg.inv(M),U)-A*np.matmul(np.linalg.inv(M),Av(Pc)))
    h=(1/D)*(C*np.matmul(np.linalg.inv(M),Av(Pc))-A*np.matmul(np.linalg.inv(M),U))
    w=g+h*rf
    
    wN=np.zeros(len(w))
    wS=np.sum(abs(w))
    for i in range(len(w)):        
        wN[i]=abs(w[i])/wS
    
    return w,wN

#wM(0,LecYSel(path))

#Simulación del cálculo de los pesos con el módelo mínima varianza

def  wmv(Pc):
    M= Mcov(Pc)
    U= np.ones(Pc.shape[0])
    C=np.matmul(np.transpose(Av(Pc)),np.linalg.inv(M),Av(Pc))
    w=(1/C)*np.matmul(np.linalg.inv(M),U)
    
    wN=np.zeros(len(w))
    wS=np.sum(abs(w))
    for i in range(len(w)):        
        wN[i]=abs(w[i])/wS
    
    return w,wN

wmv(LecYSel(path))

(array([-5.08790685e-05,  3.47115556e-04, -2.51186192e-04,  3.86711966e-04,
         4.10694337e-05,  1.19883273e-04,  3.97813842e-05, -1.72630716e-05,
        -5.82554251e-05, -3.29708541e-05,  8.51570632e-05, -9.67146332e-05,
         3.76484918e-04, -2.63249457e-04, -1.43711171e-04, -8.34711737e-05,
        -3.34917700e-05,  4.47161622e-05,  3.12694520e-04,  2.78820342e-05,
        -2.76924928e-04,  1.08322505e-05, -7.64297080e-05, -3.03348961e-04,
         1.03249328e-03,  6.88135022e-05]),
 array([0.01110525, 0.07576408, 0.05482581, 0.0844067 , 0.00896413,
        0.02616664, 0.00868299, 0.00376797, 0.01271527, 0.00719647,
        0.01858703, 0.02110967, 0.08217446, 0.05745883, 0.03136749,
        0.01821905, 0.00731017, 0.00976009, 0.06825109, 0.00608575,
        0.06044374, 0.00236433, 0.01668213, 0.06621125, 0.22535984,
        0.01501976]))

In [ ]:
y_t=Vt[:,0] #Vector de precio relativo

w_0=np.array([1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])

p_0=200    #capital inicial a invertir
p_1=p_0*np.matmul(y_t,w_0) #Valor del portafolio en el periodo 1
rho_1=p_1/p_0 -1     #Tasa de retorno
print(rho_1)
r_1=np.log(p_1/p_0)   #Tasa de retorno logarítmica
print(r_1)